*2021/01/12 Tue*

<hr>

# 9-3. 템플릿 메타 프로그래밍(template metaprogramming)

템플릿을 통해서 타입이 마치 인자인 것처럼 사용하는 것을 바로 <u>일반화 프로그래밍(generic programming)</u>이라고 함.

In [1]:
/* 나만의 std::array 구현 */
#include <iostream>

template <typename T, unsigned int N>
class Array {
    T data[N];

public:
    // 배열에 대한 레퍼런스 arr
    Array(T (&arr)[N]) {
        for (int i = 0; i < N; i++) {
            data[i] = arr[i];
        }
    }
    
    T *get_array() { return data; }
    
    unsigned int size() { return N; }
    
    void print_all() {
        for (int i = 0; i < N; i++) {
            std::cout << data[i] << ", ";
        }
        std::cout << std::endl;
    }
};

int main() {
    int arr[3] = {1, 2, 3};
    
    // 배열 wrapper 클래스
    Array<int, 3> arr_w(arr);
    
    arr_w.print_all();
}

템플릿으로 생성한 어떤 두 타입이 같은지는 아래와 같은 코드로 확인할 수 있음.

`(typeid(Array<int, 3>) == typeid(Array(<int, 5>))` -> `0`

즉, 두 타입은 다름. 서로 다른 템플릿 인자로 인스턴스화되었으므로.

`typeid`를 사용할 때는 `<typeinfo>` 헤더 파일을 추가하면 된다.

In [1]:
template <int N>
struct Int {
    static const int num = N;
    /* 왜 static const에 값을 저장하냐면,
    첫 번째로, C++ 클래스 멤버 중에서, 
    클래스 자체에서 저런 식으로 초기화를 할 수 있는 멤버의 타입은 static const 밖에 없고,
    두번째로, static const야 말로 이 클래스는 이것이다라는 의미를 가장 잘 나타내기 때문.
    */
};

// 위와 같이 하면, 아래처럼 객체를 생성하듯 타입들을 생성할 수 있다.

typedef Int<1> one;
typedef Int<2> two;

In [ ]:
#include <iostream>
#include <typeinfo>

template <int N>
struct Int {
    static const int num = N;
};

////////
template <typename T, typename U>
struct add {
    typedef Int<T::num + U::num> result;  //
};
////////

int main() {
    typedef Int<1> one;
    typedef Int<2> two;
    
    typedef add<one, two>::result three;  // 실제 덧셈을 수행하는 부분
    
    std::cout << "Addition result : " << three::num << std::endl;
    // 3이 출력
}

`one`, `two`는 객체가 아니라 1, 2의 값을 나타내는 타입이 된다.
그런데, `one`, `two`를 가지고 재미있게도 마치 `int` 변수를 다루는 것처럼 연산자를 만들 수 있다!

`typedef add<one, two>::result three;`

이 부분이 실제 덧셈을 수행하는 부분. `add` 클래스를 함수라 생각한다면, 그 계산 결과를 내부 `result` 타입으로 반환한다고 보면 된다. `one`, `two`를 더한 것을 나타내는 타입이 `result`로 정의되고, 이를 `three`라 부른다는 것!

흥미로운 것은, 3이라는 값이 런타임 시에 계산되는 것이 아니라는 것! 컴파일 타임에 3으로 덧셈이 수행되고 치환되는 것!

## 템플릿 메타 프로그래밍(Template Meta Programming - TMP)